In [1]:
import dldata.stimulus_sets.hvm as hvm

Can't import h5py
Can't import separate mcc package
Can't import asgd.


/Users/jjpr/anaconda/envs/dldata/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/jjpr/anaconda/envs/dldata/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/Users/jjpr/anaconda/envs/dldata/lib/python2.7/site-packages/sklearn/lda.py:6: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be remov

In [3]:
import xarray as xr

In [7]:
import random

In [40]:
from collections import OrderedDict

In [45]:
from pytest import approx

In [4]:
dl_hvm = hvm.HvMWithDiscfade()

In [12]:
dl_hvm_neu = dl_hvm.get_neuronal_feature_reps(sort_by_meta=True)

In [6]:
dl_meta = dl_hvm.meta

In [10]:
dl_meta_neu = dl_hvm.meta_neu

In [8]:
xr_hvm = xr.open_dataarray("/Users/jjpr/.mkgu/data/dicarlo.Hong2011/hvm_neuronal_features.nc")

In [9]:
xr_hvm

<xarray.DataArray (neuroid: 296, presentation: 268800, time_bin: 1)>
[79564800 values with dtype=float32]
Coordinates:
    neuroid_id      (neuroid) object u'Chabo_L_M_5_9' u'Chabo_L_M_6_9' ...
    arr             (neuroid) object u'M' u'M' u'M' u'M' u'M' u'M' u'M' u'M' ...
    time_bin_start  (time_bin) int64 70
    col             (neuroid) int64 9 9 8 9 8 8 7 7 5 6 4 9 9 9 9 9 8 7 9 6 ...
    hemisphere      (neuroid) object u'L' u'L' u'L' u'L' u'L' u'L' u'L' u'L' ...
    subregion       (neuroid) object u'cIT' u'cIT' u'cIT' u'cIT' u'cIT' ...
    animal          (neuroid) object u'Chabo' u'Chabo' u'Chabo' u'Chabo' ...
    y               (neuroid) float64 0.2 0.6 0.2 1.0 0.6 1.0 1.8 1.0 1.8 ...
    x               (neuroid) float64 1.8 1.8 1.4 1.8 1.4 1.4 1.0 1.0 0.2 ...
    time_bin_end    (time_bin) int64 170
    region          (neuroid) object u'IT' u'IT' u'IT' u'IT' u'IT' u'IT' ...
    row             (neuroid) int64 5 6 5 7 6 7 9 7 9 8 9 2 1 3 6 7 6 6 8 5 ...
    image_id     

In [13]:
[s.shape for s in dl_hvm_neu.values()]

[(28, 640, 296), (51, 2560, 296), (47, 2560, 296)]

In [14]:
dl_axes = ["repetition", "image", "neuroid"]

In [15]:
dl_hvm_neu.keys()

['V0', 'V3', 'V6']

In [24]:
dl_meta.dtype

dtype((numpy.record, [('obj', 'S24'), ('rxz', '<f8'), ('rxy', '<f8'), ('ryz', '<f8'), ('ty', '<f8'), ('tz', '<f8'), ('s', '<f8'), ('bg_id', 'S40'), ('size', '<i8'), ('var', 'S2'), ('_id', 'S43'), ('filename', 'S236'), ('id', 'S40'), ('category', 'S7'), ('rxz_semantic', '<f8'), ('rxy_semantic', '<f8'), ('ryz_semantic', '<f8')]))

In [26]:
dl_meta["id"][28]

'768108628bf1edd1a2db3effb28032235d6407a2'

In [27]:
dl_meta_neu.dtype

dtype((numpy.record, [('animal', 'S5'), ('region', 'S2'), ('arr', 'S1'), ('row', '<i8'), ('col', '<i8'), ('y', '<f8'), ('x', '<f8')]))

In [28]:
dl_meta_neu["animal"][151]

'Tito'

In [34]:
def get_dl_sample(dl_hvm_neu, dl_meta, dl_meta_neu):
    sample = {}
    var = random.choice(dl_hvm_neu.keys())
    sample["dl_variation"] = var
    neu = dl_hvm_neu[var]
    
    rep = sample["dl_repetition_idx"] = random.randrange(neu.shape[0])
    
    im = sample["dl_image_idx"] = random.randrange(neu.shape[1])
    sample["dl_image_id"] = dl_meta["id"][im]
    
    n = sample["dl_neuroid_idx"] = random.randrange(neu.shape[2])
    sample["dl_neuroid_animal"] = dl_meta_neu["animal"][n]
    sample["dl_neuroid_hemisphere"] = "L"
    sample["dl_neuroid_arr"] = dl_meta_neu["arr"][n]
    sample["dl_neuroid_row"] = dl_meta_neu["row"][n]
    sample["dl_neuroid_col"] = dl_meta_neu["col"][n]
    
    val = sample["dl_value"] = neu[rep, im, n]
    return sample
sample = get_dl_sample(dl_hvm_neu, dl_meta, dl_meta_neu)
sample

{'dl_image_id': '129b3dcb5eee92cc062a594fe3776bc098c957d7',
 'dl_image_idx': 441,
 'dl_neuroid_animal': 'Tito',
 'dl_neuroid_arr': 'A',
 'dl_neuroid_col': 9,
 'dl_neuroid_hemisphere': 'L',
 'dl_neuroid_idx': 173,
 'dl_neuroid_row': 5,
 'dl_repetition_idx': 2,
 'dl_value': -1.0457006,
 'dl_variation': 'V0'}

In [38]:
def coords_for_dim(xr_data, dim, exclude_indexes=True):
    result = OrderedDict()
    for key, value in xr_data.coords.variables.items():
        only_this_dim = value.dims == (dim,)
        exclude_because_index = exclude_indexes and isinstance(value, xr.IndexVariable)
        if only_this_dim and not exclude_because_index:
            result[key] = value
    return result

In [36]:
def gather_indexes(xr_data):
    """This is only necessary as long as xarray cannot persist MultiIndex to netCDF.  """
    coords_d = {}
    for dim in xr_data.dims:
        coords = coords_for_dim(xr_data, dim)
        if coords:
            coords_d[dim] = coords.keys()
    if coords_d:
        xr_data.set_index(append=True, inplace=True, **coords_d)
    return xr_data

In [41]:
xr_hvm = gather_indexes(xr_hvm)

In [43]:
xr_sample = xr_hvm.sel(image_id=sample["dl_image_id"], 
                      repetition=sample["dl_repetition_idx"],
                      animal=sample["dl_neuroid_animal"],
                      hemisphere=sample["dl_neuroid_hemisphere"],
                      arr=sample["dl_neuroid_arr"],
                      row=sample["dl_neuroid_row"],
                      col=sample["dl_neuroid_col"])
xr_sample

<xarray.DataArray (neuroid: 1, stimulus: 1, time_bin: 1)>
array([[[-1.045701]]], dtype=float32)
Coordinates:
  * stimulus        (stimulus) int64 417
  * neuroid         (neuroid) MultiIndex
  - neuroid_id      (neuroid) object u'Tito_L_A_5_9'
  - subregion       (neuroid) object u'cIT'
  - y               (neuroid) float64 0.2
  - x               (neuroid) float64 1.8
  - region          (neuroid) object u'IT'
  * time_bin        (time_bin) MultiIndex
  - time_bin_start  (time_bin) int64 70
  - time_bin_end    (time_bin) int64 170

In [57]:
xr_sample.values[0, 0, 0] == approx(sample["dl_value"], abs=1e-6)

True

In [ ]:
samples = []
for i in range(1000):
    sample = {}
    sample[""]